In [2]:
import pandas as pd
import numpy as np

# ===========================================
# 🚀 1️⃣ CARREGAR OS ARQUIVOS CSV
# ===========================================
print("\n===============================")
print("🚀 1️⃣ Carregando os arquivos...")
print("===============================")

ride = pd.read_csv('ride_v2_limpo.csv', delimiter=';', low_memory=False)
product = pd.read_csv('product_limpo.csv', delimiter=';', low_memory=False)
estimative = pd.read_csv('rideestimative_v3_limpo.csv', delimiter=';', low_memory=False)
address = pd.read_csv('rideaddress_v1_limpo.csv', delimiter=';', low_memory=False)

print(f"✅ ride.csv → {ride.shape}")
print(f"✅ product.csv → {product.shape}")
print(f"✅ estimative.csv → {estimative.shape}")
print(f"✅ address.csv → {address.shape}")

# ===========================================
# 🎯 2️⃣ PADRONIZAR NOMES DAS COLUNAS
# ===========================================
print("\n===============================")
print("🎯 2️⃣ Padronizando nomes das colunas...")
print("===============================")

ride.columns = ride.columns.str.upper().str.strip()
product.columns = product.columns.str.upper().str.strip()
estimative.columns = estimative.columns.str.upper().str.strip()
address.columns = address.columns.str.upper().str.strip()

# ✅ Padronizar valores de CITY
address['CITY'] = address['CITY'].str.upper().str.strip()

# ✅ Converter LAT e LNG para float (corrigir erro)
address['LAT'] = pd.to_numeric(address['LAT'], errors='coerce')
address['LNG'] = pd.to_numeric(address['LNG'], errors='coerce')

# ✅ Remover valores nulos após conversão
address.dropna(subset=['LAT', 'LNG'], inplace=True)

print("✅ Colunas padronizadas com sucesso!")

# ===========================================
# 🌍 3️⃣ FILTRAR POR LATITUDE E LONGITUDE (SÃO PAULO)
# ===========================================
print("\n===============================")
print("🌍 3️⃣ Filtrando por coordenadas de São Paulo...")
print("===============================")

# Definir os limites da bounding box para São Paulo
lat_min, lat_max = -24.0087, -23.3567
lon_min, lon_max = -46.8256, -46.3656

# Filtrar apenas registros dentro da bounding box
address_sp = address[
    (address['LAT'] >= lat_min) & (address['LAT'] <= lat_max) &
    (address['LNG'] >= lon_min) & (address['LNG'] <= lon_max)
]

# ✅ Mostrar tamanho após o filtro inicial
print(f"✅ Tamanho de 'address' após filtro de São Paulo: {address_sp.shape}")

# ===========================================
# 🔗 4️⃣ FAZER O MERGE (ETAPA POR ETAPA)
# ===========================================
print("\n===============================")
print("🔗 4️⃣ Fazendo o merge das tabelas...")
print("===============================")

# Primeiro merge: ride + estimative
df = ride.merge(estimative, on='RIDEID', how='inner')

# Segundo merge: incluir address (já filtrado para São Paulo)
df = df.merge(address_sp, on='RIDEID', how='inner')

# Terceiro merge: incluir product
df = df.merge(product, on='PRODUCTID', how='left')

# ✅ Mostrar tamanho após merge
print(f"✅ Tamanho após merge: {df.shape}")

# ===========================================
# 🧹 5️⃣ REMOVER COLUNAS DUPLICADAS
# ===========================================
print("\n===============================")
print("🧹 5️⃣ Removendo colunas duplicadas...")
print("===============================")

def remove_duplicate_columns(df):
    to_drop = []
    for col1 in df.columns:
        for col2 in df.columns:
            if col1 != col2 and df[col1].equals(df[col2]):
                to_drop.append(col2)
    df.drop(columns=to_drop, inplace=True)

# ✅ Remover colunas duplicadas
remove_duplicate_columns(df)

# ✅ Remover registros duplicados
df.drop_duplicates(inplace=True)

print(f"✅ Tamanho após remoção de duplicatas: {df.shape}")

# ===========================================
# 🔍 6️⃣ TRATAR VALORES NULOS
# ===========================================
print("\n===============================")
print("🔍 6️⃣ Tratando valores nulos...")
print("===============================")

if not df.empty:
    # ✅ Preencher valores nulos em colunas categóricas com 'Desconhecido'
    df['CITY'] = df['CITY'].fillna('DESCONHECIDO')
    df['NEIGHBORHOOD'] = df['NEIGHBORHOOD'].fillna('DESCONHECIDO')
    df['STATE'] = df['STATE'].fillna('DESCONHECIDO')

    # ✅ Preencher valores nulos em colunas numéricas com média
    df['FEE'] = df['FEE'].fillna(df['FEE'].mean())
    df['WAITINGTIME'] = df['WAITINGTIME'].fillna(df['WAITINGTIME'].mean())

print(f"✅ Tamanho após preenchimento de nulos: {df.shape}")

# ===========================================
# 💾 7️⃣ SALVAR RESULTADO FINAL
# ===========================================
print("\n===============================")
print("💾 7️⃣ Salvando resultado final...")
print("===============================")

arquivo_saida = 'dados_sao_paulo.csv'
df.to_csv(arquivo_saida, index=False)

print(f"\n✅ Arquivo salvo com sucesso: '{arquivo_saida}' 🎉")


🚀 1️⃣ Carregando os arquivos...
✅ ride.csv → (500000, 15)
✅ product.csv → (237, 4)
✅ estimative.csv → (2000000, 9)
✅ address.csv → (1000000, 11)

🎯 2️⃣ Padronizando nomes das colunas...
✅ Colunas padronizadas com sucesso!

🌍 3️⃣ Filtrando por coordenadas de São Paulo...
✅ Tamanho de 'address' após filtro de São Paulo: (296757, 11)

🔗 4️⃣ Fazendo o merge das tabelas...
✅ Tamanho após merge: (1356454, 36)

🧹 5️⃣ Removendo colunas duplicadas...
✅ Tamanho após remoção de duplicatas: (1356454, 36)

🔍 6️⃣ Tratando valores nulos...
✅ Tamanho após preenchimento de nulos: (1356454, 36)

💾 7️⃣ Salvando resultado final...

✅ Arquivo salvo com sucesso: 'dados_sao_paulo.csv' 🎉


In [3]:
# ✅ Mostrar preview dos dados após o merge
print("\n===============================")
print("🚀 **Preview dos dados após merge:**")
print("===============================")

# Mostrar as primeiras 10 linhas (ajuste o valor conforme necessário)
print(df.head(10))


🚀 **Preview dos dados após merge:**
    RIDEID                                USERID                     SCHEDULE  \
0  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
1  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
2  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
3  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
4  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
5  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
6  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
7  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
8  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
9  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   

                        CREATE  RIDESTATUSID  COMPANYID  PROVIDERID_x  

In [4]:
# ✅ Verificar valores 'DESCONHECIDO' em cada coluna
desconhecido_count = (df == 'DESCONHECIDO').sum()

# ✅ Exibir o número e a porcentagem de 'DESCONHECIDO' em cada coluna
desconhecido_percent = (desconhecido_count / len(df)) * 100

# ✅ Criar uma tabela formatada com os resultados
df_desconhecido = pd.DataFrame({
    'Coluna': desconhecido_count.index,
    'Total Desconhecido': desconhecido_count.values,
    '% Desconhecido': desconhecido_percent.values
})

# ✅ Exibir a tabela ordenada pela porcentagem de valores desconhecidos (decrescente)
df_desconhecido.sort_values(by='% Desconhecido', ascending=False)

,Coluna,Total Desconhecido,% Desconhecido
28,CITY,869381,64.092184
0,RIDEID,0,0.000000
26,NUMBER,0,0.000000
20,SELECTED,0,0.000000
21,RIDEREASONSELECTEDESTIMATIVEID,0,0.000000
22,FEE,0,0.000000
23,RIDEADDRESSID,0,0.000000
24,ADDRESS,0,0.000000
25,STREET,0,0.000000
27,NEIGHBORHOOD,0,0.000000


In [5]:
# ✅ Filtrar registros com CITY = 'DESCONHECIDO'
df_desconhecido_city = df[df['CITY'] == 'DESCONHECIDO'][['CITY', 'LAT', 'LNG']]

# ✅ Mostrar os primeiros registros com 'DESCONHECIDO'
print("\n🚀 **Preview de registros com CITY = 'DESCONHECIDO':**")
print(df_desconhecido_city.head(10))

# ✅ Gerar valor MÁXIMO e MÍNIMO de LAT e LNG
lat_min, lat_max = df_desconhecido_city['LAT'].min(), df_desconhecido_city['LAT'].max()
lng_min, lng_max = df_desconhecido_city['LNG'].min(), df_desconhecido_city['LNG'].max()

print("\n📌 **Valores de LAT e LNG para preenchimento:**")
print(f"➡️ Latitude mínima: {lat_min}")
print(f"➡️ Latitude máxima: {lat_max}")
print(f"➡️ Longitude mínima: {lng_min}")
print(f"➡️ Longitude máxima: {lng_max}")


🚀 **Preview de registros com CITY = 'DESCONHECIDO':**
            CITY        LAT        LNG
1   DESCONHECIDO -23.615867 -46.681102
3   DESCONHECIDO -23.615867 -46.681102
5   DESCONHECIDO -23.615867 -46.681102
7   DESCONHECIDO -23.615867 -46.681102
9   DESCONHECIDO -23.615867 -46.681102
11  DESCONHECIDO -23.615867 -46.681102
13  DESCONHECIDO -23.615867 -46.681102
15  DESCONHECIDO -23.615867 -46.681102
17  DESCONHECIDO -23.615867 -46.681102
19  DESCONHECIDO -23.615867 -46.681102

📌 **Valores de LAT e LNG para preenchimento:**
➡️ Latitude mínima: -24.0084044
➡️ Latitude máxima: -23.360978
➡️ Longitude mínima: -46.8255893
➡️ Longitude máxima: -46.3656786


In [6]:
# ✅ Preencher CITY com valor padrão se LAT e LNG estiverem dentro dos limites
lat_media = (lat_min + lat_max) / 2
lng_media = (lng_min + lng_max) / 2

df.loc[
    (df['CITY'] == 'DESCONHECIDO') &
    (df['LAT'] >= lat_min) & (df['LAT'] <= lat_max) &
    (df['LNG'] >= lng_min) & (df['LNG'] <= lng_max),
    'CITY'
] = 'SÃO PAULO - ZONA INDEFINIDA'

# ✅ Mostrar resultado após preenchimento
print("\n✅ **Preenchimento de valores desconhecidos concluído!**")
print(df[df['CITY'] == 'SÃO PAULO - ZONA INDEFINIDA'][['CITY', 'LAT', 'LNG']].head(10))


✅ **Preenchimento de valores desconhecidos concluído!**
                           CITY        LAT        LNG
1   SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102
3   SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102
5   SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102
7   SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102
9   SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102
11  SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102
13  SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102
15  SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102
17  SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102
19  SÃO PAULO - ZONA INDEFINIDA -23.615867 -46.681102


In [7]:
# ✅ Verificar valores 'DESCONHECIDO' em cada coluna
desconhecido_count = (df == 'DESCONHECIDO').sum()

# ✅ Exibir o número e a porcentagem de 'DESCONHECIDO' em cada coluna
desconhecido_percent = (desconhecido_count / len(df)) * 100

# ✅ Criar uma tabela formatada com os resultados
df_desconhecido = pd.DataFrame({
    'Coluna': desconhecido_count.index,
    'Total Desconhecido': desconhecido_count.values,
    '% Desconhecido': desconhecido_percent.values
})

# ✅ Exibir a tabela ordenada pela porcentagem de valores desconhecidos (decrescente)
df_desconhecido.sort_values(by='% Desconhecido', ascending=False)

,Coluna,Total Desconhecido,% Desconhecido
0,RIDEID,0,0.0
1,USERID,0,0.0
20,SELECTED,0,0.0
21,RIDEREASONSELECTEDESTIMATIVEID,0,0.0
22,FEE,0,0.0
23,RIDEADDRESSID,0,0.0
24,ADDRESS,0,0.0
25,STREET,0,0.0
26,NUMBER,0,0.0
27,NEIGHBORHOOD,0,0.0


In [8]:
# ✅ Verificar valores nulos em cada coluna
nulos_count = df.isnull().sum()

# ✅ Calcular a porcentagem de nulos em cada coluna
nulos_percent = (nulos_count / len(df)) * 100

# ✅ Criar DataFrame formatado com os resultados
df_nulos = pd.DataFrame({
    'Coluna': nulos_count.index,
    'Total Nulos': nulos_count.values,
    '% Nulos': nulos_percent.values
})

# ✅ Ordenar de forma decrescente para facilitar a análise
df_nulos = df_nulos.sort_values(by='% Nulos', ascending=False)

# ✅ Exibir o DataFrame formatado
df_nulos


,Coluna,Total Nulos,% Nulos
13,RIDEDRIVERLOCATIONID,1343577,99.050687
12,CAR,1343449,99.041250
7,RIDEPROVIDERID,1338326,98.663574
10,CATEGORYID_x,1334840,98.406581
6,PROVIDERID_x,1334840,98.406581
26,NUMBER,205216,15.128858
28,CITY,0,0.000000
23,RIDEADDRESSID,0,0.000000
24,ADDRESS,0,0.000000
25,STREET,0,0.000000


In [9]:
# ✅ Excluir colunas com mais de 98% de nulos
df.drop(columns=['RIDEDRIVERLOCATIONID', 'CAR', 'RIDEPROVIDERID'], inplace=True)

# ✅ Consolidar CATEGORYID_x e CATEGORYID_y
df['CATEGORYID'] = df['CATEGORYID_x'].combine_first(df['CATEGORYID_y'])
df.drop(columns=['CATEGORYID_x', 'CATEGORYID_y'], inplace=True)

# ✅ Consolidar PROVIDERID_x e PROVIDERID_y
df['PROVIDERID'] = df['PROVIDERID_x'].combine_first(df['PROVIDERID_y'])
df.drop(columns=['PROVIDERID_x', 'PROVIDERID_y'], inplace=True)

# ✅ Remover a coluna 'NUMBER' (não é relevante)
df.drop(columns=['NUMBER'], inplace=True)

# ✅ Mostrar preview após o tratamento
print("\n🚀 **Preview após tratamento de nulos:**")
print(df.head(10))

# ✅ Mostrar tamanho final após tratamento
print(f"\n✅ Tamanho final do dataframe: {df.shape}")


🚀 **Preview após tratamento de nulos:**
    RIDEID                                USERID                     SCHEDULE  \
0  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
1  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
2  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
3  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
4  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
5  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
6  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
7  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
8  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   
9  1425244  a02a6fdd-47f9-4fbd-9503-4b11466a9975  2022-06-14 20:56:40.6990432   

                        CREATE  RIDESTATUSID  COMPANYID  PRICE_x  \

In [12]:
arquivo_saida = 'dados_sao_paulo.csv'
df.to_csv(arquivo_saida, index=False)
print(f"\n✅ Arquivo CSV salvo com sucesso: '{arquivo_saida}' 🎉")


✅ Arquivo CSV salvo com sucesso: 'dados_sao_paulo.csv' 🎉
